# MySQL 8.0.40

## 0) 前提

* エンジン: **MySQL 8**
* 並び順: 任意（`ORDER BY` を付けない）
* `NOT IN` は NULL 罠のため回避
* 判定は **ID 基準**、表示は仕様どおりの列名と順序

## 1) 問題

* `MyNumbers` から **ちょうど1回だけ出現する数（single number）**のうち **最大の数**を1行で返す。存在しなければ `null` を返す。

* 入力テーブル例:

  ```
  Table: MyNumbers
  +-------------+------+
  | Column Name | Type |
  +-------------+------+
  | num         | int  |
  +-------------+------+
  -- 重複あり得る
  ```

* 出力仕様:

  ```
  +-----+
  | num |
  +-----+
  |  6  |  -- single number の最大。存在しなければ NULL
  +-----+
  ```

## 2) 最適解（単一クエリ）

> ウィンドウ関数で「出現回数」を各行に載せ、そのうち `cnt = 1` の `num` の **最大値**を投影。

```sql
WITH win AS (
  SELECT
    num,
    COUNT(*) OVER (PARTITION BY num) AS cnt
  FROM MyNumbers
)
SELECT
  MAX(num) AS num
FROM win
WHERE cnt = 1;

Runtime 392 ms
Beats 64.64%

```

* `MAX(num)` により **並び替え不要**で最大の single number を1行で取得
* single number が存在しない場合、`MAX` の母集団が空になり **`NULL` を返す**（要件どおり）

## 3) 代替解

> 集約のみで十分なサイズなら、`GROUP BY ... HAVING` → その最大値を返す。

```sql
SELECT
  MAX(num) AS num
FROM (
  SELECT num
  FROM MyNumbers
  GROUP BY num
  HAVING COUNT(*) = 1
) s;

Runtime 418 ms
Beats 39.99%

```

* `NOT IN` 不要、`NULL` でも安全
* インデックスがないと全表スキャンになる点はウィンドウ版と同様

## 4) 要点解説

* **方針**:

  1. 各 `num` の出現回数を計算（ウィンドウ or 集約）
  2. `= 1`（single）に絞る
  3. **最大値**だけを返す → `ORDER BY`・`LIMIT` 不要
* **NULL / 重複**:

  * `num` に `NULL` があっても `COUNT(*)` は `NULL` を数えるため、`num IS NULL` は single になり得る。ただし問題の意図は整数なので通常は非NULL前提。もし `NULL` 行があっても `MAX(num)` は `NULL` を無視するため影響しない。
* **安定性**:

  * 出力は1行のみで順序不要。`ORDER BY` を付けないほうが速い。

## 5) 計算量（概算）

* ウィンドウ版: `COUNT() OVER (PARTITION BY num)` は **O(N)**～**O(N log N)**（実装依存・ソート/ハッシュ）
* 集約版: `GROUP BY num` は **O(N)**～**O(N log N)**
* 推奨インデックス: `INDEX(num)` があればハッシュ/ツリー集約が効きやすい

## 6) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 MyNumbers] --> B[出現回数を算出 cnt]
  B --> C[cnt が 1 の行に絞る]
  C --> D[最大 num を求める]
  D --> E[出力 列 num だけ]
```

いい感じの結果です（特にウィンドウ版で ~65% 上回り）が、**もう少し縮められる余地**はあります。要点だけ手短に👇

---

## まずはインデックス

```sql
CREATE INDEX ix_mynumbers_num ON MyNumbers(num);
```

* `GROUP BY num` / `COUNT(*)` が**インデックス順走査**でまとまりやすくなり、
  一時テーブルやファイルソートの発生を抑制できます（環境次第で体感差が大きいところ）。

---

## 速度重視の実戦解（早期終了を効かせる）

> 並び順が任意という仕様でしたが、**パフォーマンス最優先**なら `ORDER BY ... DESC LIMIT 1` による **早期終了**が効きます。`INDEX(num)` があると特に強いです。

```sql
-- 早いことが多い版（上位1件だけ取りに行く）
SELECT num
FROM MyNumbers
GROUP BY num
HAVING COUNT(*) = 1
ORDER BY num DESC
LIMIT 1;

Wrong Answer
13 / 18 testcases passed
```

* 右端（最大値側）から**逆順インデックス走査**し、最初に見つかった「出現1回」のグループで終わるため、
  データ分布によっては **大幅短縮**します（特に「大きい値ほどユニークが出やすい」分布）。

> 出力は1行だけで、外側に `MAX` をかける必要はありません。

---

## `ORDER BY` を避けたい場合の最適形

あなたの代替解は正攻法で、インデックス追加だけでも十分効きます。書式はそのままでOK：

```sql
-- あなたの代替解（INDEXあり想定）
SELECT
  MAX(num) AS num
FROM (
  SELECT num
  FROM MyNumbers
  GROUP BY num
  HAVING COUNT(*) = 1
) s;

Runtime 396 ms
Beats 60.49%

```

* ウィンドウ版より **`GROUP BY` 直集約**のほうが MySQL では速く出ることが多いです（特に `INDEX(num)` あり）。

---

## 重複が極端に多い場合の小技（重複集合を先に作る）

> 「ほとんどが重複で、ユニークが少ない」分布なら、**重複集合だけ先に抽出**して引き算すると速いことがあります。

```sql
WITH dup AS (
  SELECT num
  FROM MyNumbers
  GROUP BY num
  HAVING COUNT(*) > 1
),
uniq AS (
  SELECT DISTINCT num
  FROM MyNumbers
  LEFT JOIN dup USING (num)
  WHERE dup.num IS NULL
)
SELECT MAX(num) AS num
FROM uniq;

Runtime 414 ms
Beats 43.42%

```

* `dup` のサイズが小さくなる分、以降の結合・探索が軽くなります（分布依存）。

---

## 実務メモ

* `EXPLAIN ANALYZE` で
  *「Using index」でグループ化できているか*、
  *一時テーブル／ファイルソートが消えているか* を確認。
* `ANALYZE TABLE MyNumbers;` で統計を更新しておくとプランが安定。
* `num` に `NULL` が混じっていても、`GROUP BY` + `HAVING COUNT(*)=1` は正しく動き、
  `MAX(num)` も `NULL` を無視するので問題ありません。

---

### まとめ

* まずは `INDEX(num)` を追加。
* 速度をさらに取りに行くなら **`GROUP BY ... HAVING COUNT(*)=1 ORDER BY num DESC LIMIT 1`**。
* `ORDER BY` を使わない方針を守るなら、あなたの **集約版 + インデックス** が最善に近いです。

